# MATLAB Kernel

This kernel is based on the official MATLAB docker image for R2023a, and the excellent [matlab_kernel](https://github.com/Calysto/matlab_kernel) package by [Calysto](https://github.com/Calysto). Note that MATLAB has recently published their own official Jupyter integration [jupyter-matlab-proxy](https://github.com/mathworks/jupyter-matlab-proxy), but the way their proxy works makes it difficult to use with a Dockerized JupyterHub setup. The other advantage of a Dockerized setup is that each notebook runs in an independent MATLAB kernel.

The key to getting the MATLAB kernel to work is ensuring each MATLAB has an appropriate license. Note that LNU licensing will not work with `matlabengine`; you actually need to specify a license file. This can be done with an environment variable in Docker.

## Dockerfile

In [ ]:
%%file matlab.dockerfile

# Use the mathworks/matlab base image, matlabengine not yet compatible with R2023b
FROM mathworks/matlab-deep-learning:r2023a

# Add software and paths as root
USER root
WORKDIR /

# Need this env variable for matlabengine
ENV LD_LIBRARY_PATH=/opt/matlab/R2023a/bin/glnxa64:$LD_LIBRARY_PATH

# Add matlabengine and matlab_kernel
RUN pip install matlabengine==9.14.3
RUN pip install matlab_kernel==0.17.1

# Copy over the passwd and group databases to allow sane permissions
COPY /etc/passwd /etc/passwd
COPY /etc/group /etc/group

# Start the matlab_kernel, prevent the container's entrypoint script from running
ENTRYPOINT []
CMD python -m matlab_kernel -f $DOCKERNEL_CONNECTION_FILE

## Build the dockerfile

In [ ]:
!docker build -t matlab:latest -f ./matlab.dockerfile /

## Install the dockerfile with dockernel

In [ ]:
!dockernel install matlab:latest \
    --name MATLAB-R2023a \
    -v labnotebook-homedirs:/home:rw \
    --gpus all \
    --network host \
    -e MLM_LICENSE_FILE=~/.matlab/matlab_license.lic \
    --user -1 \
    --group-add -1 \
    --language MATLAB

You may need to refresh your browser in order to see kernel as an option in the Launcher.